# Imports

In [57]:
import pandas as pd
import json
import validate
from general import many_to_many, many_cols
import ingest
import clean

# Pull Data into Pandas

In [58]:
data = ingest.main('Updated Inventory.xlsx')

In [59]:
excel_data = data['excel_data']
db = data['database']

# Cleaning

In [60]:
db = clean.remove_NaN_columns(db)

In [61]:
db = clean.strip_all_columns(db)

['short_name', 'type', 'parent short_name', 'foreign-campaign-short_name', 'deployment', 'description', 'region_description', 'published_list', 'reports', 'reference_file']


In [62]:
db['gcmd_phenomena']['ignore_code'] = db['gcmd_phenomena']['ignore_code'].apply(lambda x: str(x))

# Short Name Supplementation

In [63]:
# make unique shortname by combining the campaign name and the table sub short name

db['deployment']['short_name'] = db['deployment']['foreign-campaign-short_name']+'_'+db['deployment']['ignore_deployment_id']

db['collection_period'] = many_to_many(db, 'linking', 'table-instrument-short_name', keep_all=True)
db['collection_period']['short_name'] = db['collection_period']['foreign-campaign-short_name']+'_'+db['collection_period']['foreign-deployment-short_name']

# Find Matching Deployments for IOPSE

In [64]:
db['iopse']['deployment_short_name'] = 'No Matches'

for row in range(len(db['iopse'])):
    iop_start = db['iopse'].iloc[row]['start_date']
    iop_end = db['iopse'].iloc[row]['end_date']
    iop_camp = db['iopse'].iloc[row]['foreign-campaign-short_name']
    
    campaign_filter = db['deployment']['foreign-campaign-short_name'].apply(lambda short_name: short_name == iop_camp)
    possible_campaigns = db['deployment'][campaign_filter]
    
    start_filter = possible_campaigns['start_date'].apply(lambda dep_start: validate.vali_date(dep_start, iop_start))
    end_filter = possible_campaigns['end_date'].apply(lambda dep_end: validate.vali_date(iop_end, dep_end))
    
    matching_deployments = list(possible_campaigns[start_filter*end_filter]['short_name'])
    if len(matching_deployments)>1:
        print(f"error on {db['iopse'].iloc[row]}")
    elif len(matching_deployments)==1:
        matching_deployments = matching_deployments[0]
    else:
        matching_deployments = 'None Found'
    db['iopse']['deployment_short_name'].iloc[row]=matching_deployments

/home/carson/anaconda3/envs/admg/lib/python3.8/site-packages/pandas/core/computation/expressions.py:177: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(
/home/carson/anaconda3/envs/admg/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


# Campaign Filter

In [65]:
campaign_list = [
    "ACES",
    "AirMOSS",
    "ARCTAS",
    "CARVE",
    "DC3",
    "GCPEx",
    "GOES-R PLT",
    "GRIP",
    "HS3",
    "OLYMPEX"
]
db['campaign'] = db['campaign'][db['campaign']['short_name'].apply(lambda x: x in campaign_list)]

# Many to Many Creation

In [66]:
main_table_names = ['campaign', 'platform', 'instrument', 'deployment', 'iopse']

In [67]:
for table in main_table_names:
    print(table)
    for column in [col for col in db[table].keys() if isinstance(col,str) and 'table' in col]:
        name = column.split('-')[1]
        new_table_name = f"{table}-to-{name}"
        db[new_table_name]=many_to_many(db, table, column)
        print(f'   {new_table_name} created')

campaign
   campaign-to-focus_area created
   campaign-to-season created
   campaign-to-platform_type created
   campaign-to-gcmd_phenomena created
   campaign-to-repository created
   campaign-to-partner_org created
   campaign-to-gcmd_project created
platform
   platform-to-gcmd_platform created
instrument
   instrument-to-gcmd_instrument created
   instrument-to-instrument_type created
   instrument-to-measurement_keywords created
   instrument-to-geophysical_concept created
   instrument-to-repository created
   instrument-to-measurement_region created
deployment
   deployment-to-geographical_region created
iopse


# Validation

### Short Name Duplicates

In [68]:
for table_name in db.keys():
    if 'short_name' in db[table_name].keys():
        print(table_name)
        duplicates = validate.find_duplicates(db, table_name, 'short_name')

        print(f'    {duplicates}')

platform_type
    []
home_base
    []
repository
    []
focus_area
    []
season
    []
instrument_type
    []
measurement_region
    []
geographical_region
    []
geophysical_concept
    ['informationnotavailable']
campaign
    []
platform
    ['informationnotavailable']
instrument
    ['informationnotavailable', 'atlas']
deployment
    []
iopse
    []
gcmd_instrument
    ['atlas', 'informationnotavailable', 'wcr', nan, 'epic', 'opc', 'gnssreceiver', 'cris', 'aa', 'iris', 'particlespectrometers', 'ssies', 'icecube', 'aps']
gcmd_platform
    ['kingair', 'informationnotavailable', 'goes10', 'goes11', 'goes12', 'goes13', 'goes14', 'goes15', 'goes16', 'goes1', 'goes2', 'goes3', 'goes4', 'goes5', 'goes6', 'goes7', 'goes8', 'goes9', 'environmentalmodeling']
gcmd_project
    ['afsisclimate', 'camp', 'informationnotavailable', 'iodp', 'landsat7', 'mcmurdopredatorprey', 'notapplicable']
partner_org
    []
collection_period
    ['hs3dep2012', 'hs3dep2013', 'hs3dep2014', 'olympexdep2016', 'iphex

### Foriegn Key Links

### Campaign

In [69]:
errors = validate.foriegn_keys(db, 
                                data_table='campaign-to-gcmd_project', 
                                data_index='campaign', 
                                data_column='gcmd_project', 
                                foriegn_table='gcmd_project', 
                                foriegn_column='gcmd_uuid')
errors

,campaign,gcmd_project,suggestions


In [70]:
errors = validate.foriegn_keys(db, 
                                data_table='campaign-to-focus_area', 
                                data_index='campaign', 
                                data_column='focus_area', 
                                foriegn_table='focus_area', 
                                foriegn_column='short_name')
errors

,campaign,focus_area,suggestions


In [71]:
errors = validate.foriegn_keys(db, 
                                data_table='campaign-to-season', 
                                data_index='campaign', 
                                data_column='season', 
                                foriegn_table='season', 
                                foriegn_column='short_name')
errors

,campaign,season,suggestions


In [72]:
errors = validate.foriegn_keys(db, 
                                data_table='campaign-to-platform_type', 
                                data_index='campaign', 
                                data_column='platform_type', 
                                foriegn_table='platform_type', 
                                foriegn_column='short_name')
errors

,campaign,platform_type,suggestions
1,ACES,Land-based Platforms,[Land Platforms]
3,AirMOSS,Land-based Platforms,[Land Platforms]
5,ARCTAS,Land-based Platforms,[Land Platforms]
8,CARVE,Model Output,[]
9,CARVE,Land-based platforms,[Land Platforms]
10,CARVE,Maps/Charts/Photographs,[]
12,DC3,Land-based Platforms,[Land Platforms]
16,GCPEx,Land-based platforms,[Land Platforms]
17,GOES-R PLT,Satellite,[Satellites]
19,GOES-R PLT,Land-based Platforms,[Land Platforms]


In [74]:
errors = validate.foriegn_keys(db, 
                                data_table='campaign-to-gcmd_phenomena', 
                                data_index='campaign', 
                                data_column='gcmd_phenomena', 
                                foriegn_table='gcmd_phenomena', 
                                foriegn_column='ignore_code')
errors

,campaign,gcmd_phenomena,suggestions


In [75]:
errors = validate.foriegn_keys(db, 
                                data_table='campaign-to-repository', 
                                data_index='campaign', 
                                data_column='repository', 
                                foriegn_table='repository', 
                                foriegn_column='short_name')
errors

,campaign,repository,suggestions


In [76]:
errors = validate.foriegn_keys(db, 
                                data_table='campaign-to-partner_org', 
                                data_index='campaign', 
                                data_column='partner_org', 
                                foriegn_table='partner_org', 
                                foriegn_column='short_name')
errors

,campaign,partner_org,suggestions
2,ARCTAS,California Air Resources Board,[]
3,ARCTAS,International Polar Year,[]
11,HS3,Environment Canada,[]


In [77]:
errors = validate.foriegn_keys(db, 
                                data_table='platform-to-gcmd_platform', 
                                data_index='platform', 
                                data_column='gcmd_platform', 
                                foriegn_table='gcmd_platform', 
                                foriegn_column='gcmd_uuid')
errors

,platform,gcmd_platform,suggestions


In [78]:
errors = validate.foriegn_keys(db, 
                                data_table='instrument-to-measurement_keywords', 
                                data_index='instrument', 
                                data_column='measurement_keywords', 
                                foriegn_table='gcmd_phenomena', 
                                foriegn_column='ignore_code')
errors

,instrument,measurement_keywords,suggestions
98,CPL,0000,"[1000, 2000, 3000, 4000]"
147,GCAS,GEO-CAPE Airborne Simulator,[]


In [79]:
errors = validate.foriegn_keys(db, 
                                data_table='instrument-to-gcmd_instrument', 
                                data_index='instrument', 
                                data_column='gcmd_instrument', 
                                foriegn_table='gcmd_instrument', 
                                foriegn_column='gcmd_uuid')
errors

,instrument,gcmd_instrument,suggestions
8,4-STAR,b0f93e6a-c766-4957-8762-5c7709487459,[]
49,AVIRIS,d67afd03-3b79-419c-9289-5dde713ab904\n57854209...,[]
75,CIP,92f99316-b581-4adb-9980-aeb6bed64eee,[]
90,CPL,6238fe2-9a87-4e32-b866-c4a637094b51,[6238f3e2-9a87-4e32-b866-c4a637094b51]
126,EXRAD,a212d36d-2a4e-473f-b16a-6e2104b9dd8f\nba3de3fc...,[]
164,HSRL-2,abdf08cd-03c5-4497-87a4-65493584e2c7,[]
298,W-Band Radar,a90e-4a70-9bcb-93d106c1583f,[dc5ee11d-a90e-4a70-9bcb-93d106c1583f]


In [80]:
errors = validate.foriegn_keys(db, 
                                data_table='instrument-to-instrument_type', 
                                data_index='instrument', 
                                data_column='instrument_type', 
                                foriegn_table='instrument_type', 
                                foriegn_column='short_name')
errors

,instrument,instrument_type,suggestions
2,2D-C/P,Chemical Meters/Analyzers,[]
22,AirMOSS,Earth Remote Sensing - Active Remote Sensing,[]
28,AMP,In Situ/Laboratory - Spectrometers/Radiometers,[In Situ - Spectrometer/Radiometer]
35,APR-2,Earth Remote Sensing - Active Remote Sensing,[]
46,AVAPS,Profilers/Sounders,[]
87,CPL,LIDAR,[]
125,FLIR,Earth Remote Sensing - Passive Remote Sensing,[]
148,HAMSR,Spectrometers/Radiometers,[In Situ - Spectrometer/Radiometer]
153,HIRAD,Spectrometers/Radiometers,[In Situ - Spectrometer/Radiometer]
155,HIWRAP,Earth Remote Sensing - Active Remote Sensing,[]


In [81]:
errors = validate.foriegn_keys(db, 
                                data_table='instrument-to-geophysical_concept', 
                                data_index='instrument', 
                                data_column='geophysical_concept', 
                                foriegn_table='geophysical_concept', 
                                foriegn_column='short_name')
errors

,instrument,geophysical_concept,suggestions
248,RICE,Cloud Properties,[Soil Properties]
249,RICE,Processes & Dynamics,[]


In [82]:
errors = validate.foriegn_keys(db, 
                                data_table='instrument-to-repository', 
                                data_index='instrument', 
                                data_column='repository', 
                                foriegn_table='repository', 
                                foriegn_column='short_name')
errors

,instrument,repository,suggestions
88,CPL,ORNL,[]


In [83]:
errors = validate.foriegn_keys(db, 
                                data_table='instrument-to-measurement_region', 
                                data_index='instrument', 
                                data_column='measurement_region', 
                                foriegn_table='measurement_region', 
                                foriegn_column='short_name')
errors

,instrument,measurement_region,suggestions
87,CPL,full column,[]
148,HAMSR,full column,[]
153,HIRAD,boundary layer? sea surface is what I would ca...,[]
155,HIWRAP,full column,[]


In [84]:
errors = validate.foriegn_keys(db, 
                                data_table='deployment-to-geographical_region', 
                                data_index='deployment', 
                                data_column='geographical_region', 
                                foriegn_table='geographical_region', 
                                foriegn_column='short_name')
errors

,deployment,geographical_region,suggestions
94,ABoVE_dep_2017,Alaska,[]
95,ABoVE_dep_2017,western Canada,[]
96,ABoVE_dep_2018,Alaska,[]
97,ABoVE_dep_2018,western Canada,[]
98,ABoVE_dep_2019,Alaska,[]
99,ABoVE_dep_2019,western Canada,[]
100,CLAMS_dep_2001,Chesapeake Bay,[]
101,CLAMS_dep_2001,Atlantic Ocean,[]


In [85]:
errors = validate.foriegn_keys(db, 
                                data_table='deployment-to-platform', 
                                data_index='deployment', 
                                data_column='platform', 
                                foriegn_table='platform', 
                                foriegn_column='short_name')
print('\n\ndo I really need to validate this?')
errors

KeyError: 'deployment-to-platform'

In [86]:
errors = validate.foriegn_keys(db, 
                                data_table='deployment', 
                                data_index='short_name', 
                                data_column='foreign-campaign-short_name', 
                                foriegn_table='campaign', 
                                foriegn_column='short_name')
print('\n\n TODO this better once you have the data')
errors



 TODO this better once you have the data


6,short_name,foreign-campaign-short_name,suggestions
11,IPHEx_dep_2014,IPHEx,[]
18,ATom_dep_2016,ATom,[]
19,ATom_dep_2017a,ATom,[]
20,ATom_dep_2017b,ATom,[]
21,ATom_dep_2018,ATom,[]
22,SEAC4RS_dep_2013,SEAC4RS,[]
23,BOREAS_dep_1994,BOREAS,[]
24,BOREAS_dep_1996,BOREAS,[]
60,CAMEX_dep_1993,CAMEX,[]
61,CAMEX_dep_1995,CAMEX,[]


In [87]:
errors = validate.foriegn_keys(db, 
                                data_table='platform', 
                                data_index='short_name', 
                                data_column='foreign-platform_type-short_name', 
                                foriegn_table='platform_type', 
                                foriegn_column='short_name')
errors

2,short_name,foreign-platform_type-short_name,suggestions
8,ASO,Prop Plane,[]


In [88]:
errors = validate.foriegn_keys(db, 
                                data_table='platform-to-gcmd_platform', 
                                data_index='platform', 
                                data_column='gcmd_platform', 
                                foriegn_table='gcmd_platform', 
                                foriegn_column='gcmd_uuid')
errors

,platform,gcmd_platform,suggestions


In [89]:
db['iopse']

2,short_name,type,parent short_name,foreign-campaign-short_name,deployment,start_date,end_date,description,region_description,published_list,reports,reference_file,deployment_short_name
3,HS3_SE_1,SE,none,HS3,dep_2014,2014-09-11,2014-09-19,Hurricane Edouard - a period of apparent rapi...,Atlantic,https://journals.ametsoc.org/doi/pdf/10.1175/B...,Flight Reports: http://dx.doi.org/10.5067/HS3/...,Information Not Available,HS3_dep_2014
4,HS3_SE_2,SE,none,HS3,dep_2012,2012-09-11,2012-09-15,Hurricane Nadine - SAL interaction,Atlantic,https://journals.ametsoc.org/doi/pdf/10.1175/B...,Flight Reports: http://dx.doi.org/10.5067/HS3/...,Information Not Available,HS3_dep_2012
5,HS3_SE_3,SE,none,HS3,dep_2014,2014-10-15,2014-10-17,Hurricane Gonzalo - study of eyewall replacem...,Atlantic,https://journals.ametsoc.org/doi/pdf/10.1175/B...,Flight Reports: http://dx.doi.org/10.5067/HS3/...,Information Not Available,HS3_dep_2014
6,OLYMPEX_IOP_1,IOP,none,OLYMPEX,dep_2016,2015-11-10,2015-12-21,(Almost) all instruments operating,Olympic Peninsula,https://journals.ametsoc.org/doi/pdf/10.1175/B...,Flight Reports: http://dx.doi.org/10.5067/GPMG...,Information Not Available,OLYMPEX_dep_2015
7,OLYMPEX_SE_1,SE,OLYMPEX_iop_1,OLYMPEX,dep_2016,2015-11-13,2015-11-17,Atmospheric river - consecutive events,Olympic Peninsula,https://journals.ametsoc.org/doi/pdf/10.1175/B...,Science Summary: https://ghrc.nsstc.nasa.gov/p...,Information Not Available,OLYMPEX_dep_2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,TCSP_IOP_1,IOP,none,TCSP,dep_2005,2005-07-01,2005-07-27,"Studied tropical cyclone structure, formation,...","Costa Rica, Caribbean, Gulf of Mexico, eastern...",https://journals.ametsoc.org/doi/pdf/10.1175/B...,Information Not Available,Information Not Available,TCSP_dep_2005
241,TCSP_SE_1,SE,TCSP_IOP_1,TCSP,dep_2005,2005-07-05,2005-07-09,Tropical cyclone Dennis transition from TD to ...,"Caribbean, Gulf of Mexico,",https://journals.ametsoc.org/doi/pdf/10.1175/B...,https://ghrc.nsstc.nasa.gov/home/field-campaig...,Information Not Available,TCSP_dep_2005
242,TCSP_SE_2,SE,TCSP_IOP_1,TCSP,dep_2005,2005-07-15,2005-07-16,Pregenesis in the eastern Northern Pacific. Co...,eastern Northern Pacific,https://journals.ametsoc.org/doi/pdf/10.1175/B...,https://ghrc.nsstc.nasa.gov/home/field-campaig...,Information Not Available,TCSP_dep_2005
243,TCSP_SE_3,SE,TCSP_IOP_1,TCSP,dep_2005,2005-07-17,2005-07-17,Category 4 Hurricane Emily,Caribbean,https://journals.ametsoc.org/doi/pdf/10.1175/B...,https://ghrc.nsstc.nasa.gov/home/field-campaig...,Information Not Available,TCSP_dep_2005


In [90]:
errors = validate.foriegn_keys(db, 
                                data_table='iopse', 
                                data_index='iopse_id', 
                                data_column='deployment_short_name', 
                                foriegn_table='deployment', 
                                foriegn_column='short_name')
errors

KeyError: "['iopse_id'] not in index"

In [91]:
errors = validate.foriegn_keys(db, 
                                data_table='iopse', 
                                data_index='iopse_id', 
                                data_column='event_type', 
                                foriegn_table='deployment', 
                                foriegn_column='short_name')
errors

KeyError: 'event_type'

### Flight

In [92]:
# flight table, instruments

errors = validate.foriegn_keys(db, 
                               data_table='collection_period', 
                              data_index='foreign-campaign-short_name', 
                              data_column='instrument', 
                              foriegn_table='instrument', 
                              foriegn_column='short_name')
errors

,foreign-campaign-short_name,instrument,suggestions
27,IPHEx,2D-C,[2D-C/P]
34,IPHEx,Nevzorov,[Nevzorov probe]
35,IPHEx,King hot wire,[King hot wire probe]
130,SEAC4RS,Dew Point,[]
131,SEAC4RS,LWC/TWC,[]
132,SEAC4RS,Rosemount temperature,[]
133,SEAC4RS,Rosemount icing rod,[]
135,SEAC4RS,NMASS,[MAS]
140,ATom,CAPS Vienna,[]
164,ATom,CAPS Vienna,[]


In [93]:
# Flight table, platforms

errors = validate.foriegn_keys(db, 
                      data_table='collection_period', 
                      data_index='foreign-campaign-short_name', 
                      data_column='platform', 
                      foriegn_table='platform', 
                      foriegn_column='short_name')
errors

,foreign-campaign-short_name,platform,suggestions
15,HS3,WB-57f,[WB-57]
16,HS3,WB-57f,[WB-57]
19,OLYMPEX,UND Citation II,[Citation]
21,OLYMPEX,Field_Site,[Field_Sites?]
1021,HyMEx,DO128,[DO-228]
1022,HyMEx,DO128,[DO-228]
1023,HyMEx,DO128,[DO-228]
1024,HyMEx,DO128,[DO-228]
1025,HyMEx,DO128,[DO-228]
1026,HyMEx,DO128,[DO-228]


In [94]:
errors = validate.foriegn_keys(db, 
                      data_table='collection_period', 
                      data_index='foreign-campaign-short_name', 
                      data_column='foreign-deployment-short_name', 
                      foriegn_table='deployment', 
                      foriegn_column='short_name')
errors

,foreign-campaign-short_name,foreign-deployment-short_name,suggestions
0,HS3,dep_2012,"[HS3_dep_2012, DC3_dep_2012]"
1,HS3,dep_2012,"[HS3_dep_2012, DC3_dep_2012]"
2,HS3,dep_2012,"[HS3_dep_2012, DC3_dep_2012]"
3,HS3,dep_2013,[HS3_dep_2013]
4,HS3,dep_2013,[HS3_dep_2013]
...,...,...,...
1051,TCSP,dep_2005,[TCSP_dep_2005]
1052,TCSP,dep_2005,[TCSP_dep_2005]
1053,TCSP,dep_2005,[TCSP_dep_2005]
1054,TCSP,dep_2005,[TCSP_dep_2005]


# Dates

In [95]:
for table_name in db.keys():
    # find tables that have date fields
    if len(col_names := [col for col in db[table_name].columns if 'date' in col])>0:
        print(f"{table_name}\n    {', '.join(col_names)}")

campaign
    start_date, end_date, ignore_metadata_date
deployment
    start_date, end_date
iopse
    start_date, end_date


In [96]:
table_names = ['campaign', 'deployment', 'iopse'] 
for table_name in table_names:
    print(table_name)
    db[table_name]['valid_date']=False
    
    db[table_name]['valid_date'] = db[table_name].apply(lambda row: validate.vali_date(row['start_date'], row['end_date']), axis=1)
        

campaign
deployment
iopse


In [97]:
# this field no longer exists?

# table_names = ['instrument'] 
# for table_name in table_names:
    
#     db[table_name]['valid_date']=False
#     db[table_name]['valid_date'] = db[table_name].apply(lambda row: validate.vali_date(row['deployment_date'], row['decommision_date']), axis=1)
       

In [98]:
db['campaign'][['short_name','start_date','end_date','valid_date']][db['campaign']['valid_date'].apply(lambda x: not(x))]

,short_name,start_date,end_date,valid_date


# IOPSE

In [51]:
# FILTER 
# TODO: FIX GRIP
db['iopse']=db['iopse'][db['iopse']['iopse_id']!='Information Not Available'][db['iopse']['foreign-campaign-short_name']!='GRIP'][db['iopse']['foriegn_campaign_short_name']!='ATTREX']


KeyError: 'iopse_id'

In [52]:
# split into two tables

db['significant_event'] = db['iopse'][db['iopse']['event_type']=='Y'].copy()
db['iop'] = db['iopse'][db['iopse']['event_type']!='Y'].copy()

KeyError: 'event_type'

In [53]:
# validate that all iops are unique

db['iop']['iop_short_name'][db['iop']['iop_short_name'].duplicated()]

KeyError: 'iop'

In [54]:
# if sig event has matching iop, link them

# if sig event doesn't have matching iop, delete it so there will be no foriegn key

In [46]:
# tag significant events that have IOPs

db['significant_event']['has_iop'] = db['significant_event']['iop_short_name'].apply(lambda short_name: short_name in list(db['iop']['iop_short_name']))


KeyError: 'significant_event'

In [47]:
# # check that all significant events with an iop have an iop

# s_i = set(db['iop']['iop_short_name'])
# s_s = set(has_iop['iop_short_name'])
# [s for s in s_s if s not in s_i]

In [48]:
from datetime import datetime
# sig event start >= iop start

def start_val(sig_row, db):
    large_date = sig_row['start_date']
    small_date = db['iop'][db['iop']['iop_short_name']==sig_row['iop_short_name']]['start_date'].iloc[0]
    val = large_date >= small_date
    return val

def end_val(sig_row, db):
    large_date = sig_row['end_date']
    small_date = db['iop'][db['iop']['iop_short_name']==sig_row['iop_short_name']]['end_date'].iloc[0]
    val = large_date <= small_date
    return val

has_iop = db['significant_event'][db['significant_event']['has_iop']].copy()
val_iop_date_start = has_iop.apply(lambda row: not(start_val(row, db)), axis=1)
val_iop_date_end = has_iop.apply(lambda row: not(end_val(row, db)), axis=1)

KeyError: 'significant_event'

In [49]:
# display incorrect start dates
has_iop[val_iop_date_start]

NameError: name 'has_iop' is not defined

In [50]:
# display incorrect end dates
has_iop[val_iop_date_end]

NameError: name 'has_iop' is not defined

## Remap Limited Fields  I SHOULD DELETE THIS IT RUNS ELSEWHERE

### Pickle the Data

In [99]:
import pickle

In [100]:
pickle.dump(db, open('ingested_data','wb'))

In [ ]:
db['geographical_region']